In [1]:
!pip install pyspark
!pip install requests

In [2]:
#imports
from pyspark.sql import SparkSession
import os
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import requests

In [3]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell"

In [4]:
#criar sparksession
spark = SparkSession.builder \
    .appName("Conexão_MinIO") \
    .config("spark.hadoop.fs.s3a.access.key", "ueVk36Brq2ZX8gCGGj1w") \
    .config("spark.hadoop.fs.s3a.secret.key", "12345678") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://172.29.8.228:9000") \
    .getOrCreate()

In [5]:
API_KEY = "c478634695f1e63e882ce6444d5198e4"
nome_cidade = ["Aracaju","Lagarto","Itabaiana"]

In [6]:
#função para convertar já que está puxando em kelvin.
def kelvin_to_celsius(kelvin_temp):
    return kelvin_temp - 273.15

In [7]:
#decidir definir um esquema
schema = StructType([
    StructField("city", StringType(), True),  
    StructField("temp", DoubleType(), True),   
])

In [8]:
#puxar url
dados_lista = []
def get_data(city):
    response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na obtenção dos dados para {city}: {response.status_code}")
        return None

In [9]:
#continuação
for cidade in nome_cidade:
    data = get_data(cidade)
    if data:
        dados_lista.append(data)

In [10]:
df = spark.createDataFrame(dados_lista, schema)
df = df.withColumn("temp", F.format_number(df["temp"], 1))

In [11]:
bucket_path = "s3a://desafiotempo"
df.write.format("parquet") \
    .mode("overwrite") \
    .option("path", f"{bucket_path}/temperaturas.parquet") \
    .save()

In [12]:
spark.stop()